In [2]:
import json
import pandas as pd
import glob
import numpy as np
from tqdm.auto import tqdm

In [3]:
data_in = "../../sih-raw-data/geojson/geojson-cat-blocks/"
data_out = "../data/geojson/"

In [3]:
first = True
compiled_json = ""
for f in tqdm(glob.glob(data_in + "/*")[:]):
    with open(f) as file:
        block_cat_json = json.load(file)
    for i in block_cat_json["features"]:
        i["id"] = f.split("/")[-1].split(".")[0]
    if first:
        compiled_json = block_cat_json
        first = False
    else:
        compiled_json["features"].append(block_cat_json["features"][0])
    #     if not first:
    # #         print(compiled_json, first)
    #         break
with open("../data/compiled_cat_block_id.json", "w") as out:
    json.dump(compiled_json, out)

### Encode categories

In [4]:
df_cat = pd.read_parquet(r"../data/comp/categories.parquet.gzip")

In [5]:
df_cat.loc[:,'category'].unique()

array(['safe', 'saline', 'over_exploited', 'semi-critical', 'critical',
       'over-exploited', 's afe'], dtype=object)

In [6]:
df_cat=df_cat.replace({'over_exploited':'over-expoilted','s afe':'safe'})

In [7]:
categories=list(df_cat.loc[:,'category'].unique())

In [8]:
encode_cat={categories[i]:i for i in range(len(categories))}

In [9]:
encode_cat

{'safe': 0,
 'saline': 1,
 'over-expoilted': 2,
 'semi-critical': 3,
 'critical': 4,
 'over-exploited': 5}

In [10]:
df_cat['cat_val']=df_cat['category'].replace(encode_cat)

In [11]:
df_cat.head()

,state,district,block,category,cat_val
0,andaman & nicobar,n & m andaman,porlob island,safe,0
1,andaman & nicobar,n & m andaman,north passage island,safe,0
2,andaman & nicobar,n & m andaman,interview island,safe,0
3,andaman & nicobar,n & m andaman,stewart island,safe,0
4,andaman & nicobar,n & m andaman,narcondam island,safe,0


In [12]:
df_cat.to_csv(r'../data/csv/categories.csv',index=False)
df_cat.to_parquet(r'../data/comp/categories.parquet.gzip',compression='gzip',index=False)

### Remove point values

In [4]:
df = pd.read_csv(r"../data/blocks_not_found_categories.csv")

In [6]:
import shutil

first = True
poly = []
compiled_json = ""
for f in tqdm(glob.glob(data_in + "/*")):
    with open(f) as file:
        block_cat_json = json.load(file)
    if block_cat_json["features"][0]["geometry"]["type"] == "Point":
        poly.append(f.split("/")[-1])
        df = df.append(
            df_cat[
                df_cat["block"] == f.split("/")[-1].split(".")[0].strip()
            ].loc[:, ["state", "block"]],
            ignore_index=True,
        )
        shutil.move(
            f,
            "../../sih-raw-data/geojson/geojson-cat-blocks-point/"
            + f.split("/")[-1],
        )

In [12]:
df.shape

(4351, 3)

In [13]:
df.to_csv(r"../data/blocks_not_found_categories.csv", index=False)

### Process

In [27]:
geometry_types = []
for f in tqdm(glob.glob(data_in + "/*")):
    with open(f) as file:
        block_cat_json = json.load(file)
    geometry_types.append(block_cat_json["features"][0]["geometry"]["type"])

In [28]:
list(set(geometry_types))

['MultiPolygon', 'Polygon']